In [8]:
import numpy as np
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
model_iMM904 = read_sbml_model('iMM904.xml')
phy = Metabolite(id='phy', name='phytoene', compartment='c') #Adding the metabolite
new_reaction1 = Reaction('phy_syn') #PHYtoene SYNtase
phy = Metabolite(id='phy', name='phytoene', compartment='c') #Adding the metabolite

new_reaction1.add_metabolites({model_iMM904.metabolites.ggdp_c: -2,
                              phy: 1, # our newly created metabolites
                               model_iMM904.metabolites.ppi_c: 2
                                })
model_iMM904.add_reactions([new_reaction1])
new_reaction2 = Reaction('neu_syn') #NEUrosporene SYNtase
#Adding the metabolite
neu = Metabolite(id='neu', name='Neurosporene', compartment='c') 
#AH2 = Metabolite(id='AH2', name='Reduced acceptor', compartment='c') 
#A = Metabolite(id='A', name='Acceptor', compartment='c') 
new_reaction2.add_metabolites({
    model_iMM904.metabolites.nadp_c: -3,
    phy: -1, # our newly created metabolites
    model_iMM904.metabolites.nadph_c: 3,
    neu: 1,
    })
model_iMM904.add_reactions([new_reaction2])
new_reaction3 = Reaction('lyc_syn') #LYCopene SYNtase
lyc = Metabolite(id='lyc', name='Lycopene', compartment='c') #Adding the metabolite
new_reaction3.add_metabolites({
    neu: -1, # our newly created metabolites
    model_iMM904.metabolites.nadph_c: -1, 
    model_iMM904.metabolites.o2_c: -1,
    lyc: 1,
    model_iMM904.metabolites.nadp_c: 1,
    model_iMM904.metabolites.h2o_c: 2,
    })
model_iMM904.add_reactions([new_reaction3])
new_reaction4 = Reaction('beta_syn') #BETA-carotene SYNtase
beta = Metabolite(id='beta', name='Beta-carotene', compartment='c') #Adding the metabolite
new_reaction4.add_metabolites({
    lyc: -1, # our newly created metabolites
    beta: 1,
    })
model_iMM904.add_reactions([new_reaction4])
new_reaction5 = Reaction('EX_beta') #BETA-carotene SYNtase
new_reaction5.add_metabolites({
    beta: -1 #Excretion of beta-carotene
    })
model_iMM904.add_reactions([new_reaction5])
with model_iMM904:
    model_iMM904.objective = model_iMM904.reactions.EX_beta 
    print("Maximum beta-carotene flow: ",model_iMM904.optimize().objective_value)

Maximum beta-carotene flow:  0.35143769968051247


In [9]:
model_iMM904.optimize()

,fluxes,reduced_costs
CITtcp,0.000000e+00,-0.000000e+00
13BGH,0.000000e+00,-4.751566e-02
13BGHe,0.000000e+00,0.000000e+00
13GS,3.266700e-01,0.000000e+00
16GS,0.000000e+00,-9.503132e-02
...,...,...
phy_syn,-3.972329e-14,0.000000e+00
neu_syn,-3.972329e-14,-5.551115e-17
lyc_syn,-3.972329e-14,0.000000e+00
beta_syn,-3.972329e-14,0.000000e+00


In [11]:
solution = model_iMM904.optimize()

In [17]:
max_growth = model_iMM904.optimize().objective_value

In [14]:
solution.fluxes['EX_glc__D_e']

-10.0

In [18]:
print("Max growth rate on glucose:",round(max_growth,2))

Max growth rate on glucose: 0.29


In [21]:
max_growth/abs(solution.fluxes["EX_glc__D_e"])

0.028786570370404923

In [7]:
#Find the max growth rate of S. cerevisiae on glucose with the standard medium settings with glucose
#Test the maximum beta-carotene production at another growth rate
print("Max growth and production in glucose. Growth rate and production is independent:\n")
with model_iMM904: 
    #Max growth
    print(model_iMM904.objective)
    model_iMM904.optimize()
    model_iMM904.optimize().objective_value
    #solution = model_iMM904.optimize()
    #print("Max growth rate on glucose:",round(initial_growth,2))
    #print("hej", solution)
    #print(type(solution.fluxes))
    
    

Max growth and production in glucose. Growth rate and production is independent:

Maximize
1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090


In [ ]:
with model_iMM904:
    #max production
    model_iMM904.objective = model_iMM904.reactions.EX_beta           #sets objective to the beta-carotene flow
    initial_beta_flow = model_iMM904.optimize().objective_value       #Find max beta-carotene flow
    print("Max beta-carotene flow on glucose:",round(initial_beta_flow,2))
    print(initial_beta_flow/abs(initial_beta_flow.fluxes["EX_glc__D_e"]))